# Cross-referencing CUI associations with PMIDs

In [1]:
from joblib import Parallel, delayed

import pickle
import json

from tqdm import tqdm

from itertools import combinations

import pandas as pd
import numpy as np
import itertools
import random
from collections import defaultdict, Counter
from matplotlib import pyplot as plt

from pathlib import Path

In [2]:
ncores=16

In [3]:
tqdm.pandas()

## Merging CUI pairs obtained from different DBs

In [27]:
cui_pairs_fpath = Path(
    '../benchmark_data/01_cui_pairs_json/'
)

In [28]:
cui_pairs_flist = list(cui_pairs_fpath.glob('*.json'))
cui_pairs_flist

[PosixPath('../benchmark_data/01_cui_pairs_json/ctd_cui_pairs.json'),
 PosixPath('../benchmark_data/01_cui_pairs_json/gwas_cui_pairs.json'),
 PosixPath('../benchmark_data/01_cui_pairs_json/drugcentral_cui_pairs.json'),
 PosixPath('../benchmark_data/01_cui_pairs_json/mentha_cui_pairs.json'),
 PosixPath('../benchmark_data/01_cui_pairs_json/rxnav_cui_pairs.json'),
 PosixPath('../benchmark_data/01_cui_pairs_json/stringdb_cui_pairs.json'),
 PosixPath('../benchmark_data/01_cui_pairs_json/kegg_cui_pairs.json'),
 PosixPath('../benchmark_data/01_cui_pairs_json/disgennet_cui_pairs.json')]

In [29]:
all_lookup_pairs_dict = dict()

for fname in tqdm(cui_pairs_flist):
    db_name = fname.stem.split('_')[0]
    
    with open(fname, 'r') as f:
        db_cui_pairs_raw = json.load(f)
    db_cui_pairs_str_set = {
        ' '.join(sorted(p)) for p in db_cui_pairs_raw
    }
    all_lookup_pairs_dict[db_name] = db_cui_pairs_str_set

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:42<00:00, 12.79s/it]


In [9]:
{k:len(v) for k,v in all_lookup_pairs_dict.items()}

{'ctd': 9754308,
 'gwas': 642544,
 'drugcentral': 2900561,
 'mentha': 344037,
 'rxnav': 1202410,
 'stringdb': 349357,
 'kegg': 35236404,
 'disgennet': 1064881}

In [11]:
all_lookup_pairs_set = set.union(*all_lookup_pairs_dict.values())
len(all_lookup_pairs_set)

50451953

In [13]:
pd.to_pickle(
    all_lookup_pairs_set,
    '../benchmark_data/02_pubmed_pmid_cuis/all_lookup_pairs_set.pkl'
)

## Cross-referencing CUI pairs extracted from PubMed abstracts

In [4]:
all_lookup_pairs_set = pd.read_pickle(
    '../benchmark_data/02_pubmed_pmid_cuis/all_lookup_pairs_set.pkl'
)

Input: files looking like this:

In [5]:
! less ../benchmark_data/02_pubmed_pmid_cuis/ts_pmid_cuis_lines/ts_pmid_chuis_lines_part_022 | head -10

2008-05:18508489:2:C1883559 C0030956 C0442821 C0558295 C1825746 C0445022 C0439175 C0205360 C1704972 C0445022 C0037231 C1419633
2008-05:18508489:3:C0596988 C1825746 C0445022 C0037231 C0678594 C0205250 C1549865 C0445022
2008-05:18508489:4:C1704675 C0449774  C1825746 C1825746 C0445022 C1883559
2008-05:18508489:5:C1274040 C0680240 C0439064 C1517586 C0302523 C0443288 C1420510 C0017446 C0181687 C1883559 C0030956 C0205172 C0596988
2008-05:18508489:6:C1274040 C1279919 C0037231 C0678594 C3898777 C0205103 C0596973 C0333562 C1948066 C1517586 C4522252 C0596988 C0333562 C1948066
2008-05:18508490:0:C3871154 C0282554 C0017658
2008-05:18508490:1:C0023516 C0332448 C0205224 C3871154 C0543483 C0017658
2008-05:18508490:2:C0031847 C0035820 C0282554 C0597357 C1522508 C1521991 C0441712 C0023516 C0205210 C1510544 C0017658
2008-05:18508490:3:C0332448 C0023516 C0022663 C0007584 C0022663 C1417247
2008-05:18508490:4:C1517331 C0332185 C0282554 C4255326 C1326500 C0019868 C1527148 C1545588 C0007584


In [6]:
pmid_cuis_flist = (
    sorted(
        Path(
            '../benchmark_data/02_pubmed_pmid_cuis/ts_pmid_cuis_lines/'
        ).glob('*')
    )
)
len(pmid_cuis_flist)

34

In [7]:
pmid_cuis_flist[:3]

[PosixPath('../benchmark_data/02_pubmed_pmid_cuis/ts_pmid_cuis_lines/pmid_terms_2022_formatted_filt_list.txt'),
 PosixPath('../benchmark_data/02_pubmed_pmid_cuis/ts_pmid_cuis_lines/pmid_terms_addendum_2021_formatted_filt_list.txt'),
 PosixPath('../benchmark_data/02_pubmed_pmid_cuis/ts_pmid_cuis_lines/ts_pmid_chuis_lines_part_000')]

In [8]:
def open_ts_pmid_cuis_as_df(fpath):
    
    ts_pmid_cuis_df = pd.read_table(
        fpath,
        delimiter=':',
        header=None
    )
    ts_pmid_cuis_df.columns = ['ts', 'pmid', 'sent_idx', 'terms']
    
    return ts_pmid_cuis_df

In [9]:
with Parallel(n_jobs = ncores) as parallel:
    ts_pmid_cuis_df_list = parallel(
        delayed(open_ts_pmid_cuis_as_df)(i) for i in tqdm(pmid_cuis_flist)
    )

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:12<00:00,  2.76it/s]


In [10]:
ts_pmid_cuis_df = pd.concat(ts_pmid_cuis_df_list)
ts_pmid_cuis_df = ts_pmid_cuis_df.dropna()

In [11]:
ts_pmid_cuis_df

,ts,pmid,sent_idx,terms
0,2022-02,35407114,0,C0456205 C1707391 C0205198 C0043188 C0011175 C...
1,2022-02,35407114,1,C0453447 C0043188 C0011175 C0262568 C1512080 C...
2,2022-02,35407114,2,C2744535 C2603343 C1527178 C0220806 C1521970 C...
3,2022-02,35407114,3,C0043188 C0020205 C0008946 C0348080
4,2022-02,35407114,4,C0020205 C2346866
...,...,...,...,...
2886651,2012-11,23916486,12,C0028377 C0439962 C3161035 C2698651 C0237753 C...
2886652,2012-11,23916486,13,C1515021 C0549177 C1704743 C0017446 C1515021 C...
2886653,2012-11,23916486,14,C1274040 C0085862 C0680398 C1550501
2886654,2012-11,23916486,15,C0009085 C1079230 C3264163 C0348014 C0920367 C...


In [21]:
%%time 

pmid_to_ts_dict = (
    ts_pmid_cuis_df[['ts', 'pmid']]
        .drop_duplicates()
        .groupby('pmid')
        .agg(min)
        ['ts']
        .to_dict()
)

CPU times: user 31min 54s, sys: 45.7 s, total: 32min 40s
Wall time: 32min 41s


In [23]:
len(pmid_to_ts_dict)

29986221

In [22]:
with open('../benchmark_data/02_pubmed_pmid_cuis/pmid_to_ts_dict.json', 'w') as f:
    json.dump(pmid_to_ts_dict, f)

In [12]:
%%time

pmid_cuis_abstr_df = (
    ts_pmid_cuis_df[['pmid', 'terms']]
        .groupby('pmid')
        .agg(
            {
                'terms': lambda x: ' '.join(x)
            }
        )
)

CPU times: user 8min 17s, sys: 36.2 s, total: 8min 54s
Wall time: 8min 54s


In [26]:
pmid_cuis_abstr_df

,terms
pmid,
1,C0016572 C1510438 C0005889 C0392621
2,C4684445 C1522508 C0037949 C0597331 C0009235 C...
3,C0025552 C0457555 C0022023 C0182400 C2603343
4,C1280500 C0008269 C0016030 C0030685 C0521450 C...
5,C0567415 C3161035 C1305923 C0037949 C0067056 C...
...,...
36475580,C0032821 C1138408 C4019020 C0450363 C0678594 C...
36475581,C0071360 C0283001 C0700106 C0058372 C0017479 C...
36475582,C0596084 C0181074 C0596383 C0040223 C4544674 C...


### Splitting dataset into multiple chunks for parallel processing

In [4]:
df_chunks_save_path = Path(
    '../benchmark_data/02_pubmed_pmid_cuis/pmid_cui_dfs/'
)
df_chunks_save_path.mkdir(exist_ok=True, parents=True)

In [36]:
n_df_chunks = 32

In [39]:
for n in tqdm(range(n_df_chunks)):
    n_str = str(n).zfill(2)
    df_fname = f'pmid_cuis_abstr_df_part_{n_str}.pkl'
    pmid_cuis_abstr_df[:][n::n_df_chunks].to_pickle(
        df_chunks_save_path.joinpath(df_fname)
    )

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:37<00:00,  1.16s/it]


In [5]:
df_chunks_flist = sorted(df_chunks_save_path.glob('*.pkl'))
len(df_chunks_flist)

32

### Parallel processing

In [13]:
def find_coocs_chunk(chunk_fpath, coocs_fpath):
    
    chunk_df = pd.read_pickle(chunk_fpath)
    
    all_lookup_pairs_set = pd.read_pickle(coocs_fpath)
    
    def find_coocs(row):
    
        abstr_coocs = {
            ' '.join(sorted(pair))
            for pair in combinations((set(row.split(' '))), 2)
        }

        dbs_intersect = all_lookup_pairs_set.intersection(abstr_coocs)

        return dbs_intersect
    
    chunk_df['bench_cooc'] = (
        chunk_df['terms'].apply(find_coocs)
    )
    
    chunk_df = chunk_df[
        chunk_df['bench_cooc'].apply(
            lambda x: True if len(x) > 1 else False 
        )
    ]
    
    return chunk_df

In [15]:
with Parallel(n_jobs = ncores, batch_size=1) as parallel:
    pmid_cuis_bench_cooc_df_list = parallel(
        delayed(find_coocs_chunk)(
            chunk_fpath,
            '../benchmark_data/02_pubmed_pmid_cuis/all_lookup_pairs_set.pkl'
        ) for chunk_fpath in tqdm(df_chunks_flist)
    )

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [06:42<00:00, 12.58s/it]


In [16]:
pmid_cuis_bench_cooc_df = pd.concat(pmid_cuis_bench_cooc_df_list)

In [19]:
pmid_cuis_bench_cooc_df = (
    pmid_cuis_bench_cooc_df
        .reset_index()
        .explode('bench_cooc')
        .drop('terms', axis=1)
        .astype(str)
        .rename(columns={'bench_cooc': 'pair', 'pmid': 'abstr_lvl_cooc'}
)

In [18]:
with open('../benchmark_data/02_pubmed_pmid_cuis/pmid_to_ts_dict.json', 'r') as f:
    pmid_to_ts_dict = json.load(f)

## Adding source DB information

In [25]:
pmid_cuis_bench_cooc_df['date'] = pmid_cuis_bench_cooc_df['pmid'].progress_apply(lambda x: pmid_to_ts_dict.get(x))

100%|█████████████████████████████████████████████████████████████████| 57014889/57014889 [00:46<00:00, 1213498.98it/s]


In [34]:
pmid_cuis_bench_cooc_df

,abstr_lvl_cooc,pair,date
0,68,C0048581 C0134796,1975-11
0,68,C0017725 C0043047,1975-11
1,107,C0003818 C0302583,1975-10
1,107,C0009968 C0024706,1975-10
1,107,C0023175 C0043047,1975-10
...,...,...,...
7202663,36475470,C0009241 C0338656,2022-12
7202663,36475470,C0038454 C0338656,2022-12
7202663,36475470,C0009241 C0948008,2022-12
7202663,36475470,C0038454 C0948008,2022-12


In [35]:
for db_name, db_pairs in tqdm(all_lookup_pairs_dict.items()):
    pmid_cuis_bench_cooc_df[f'from_{db_name}'] = pmid_cuis_bench_cooc_df['pair'].apply(lambda x: x in db_pairs)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:52<00:00, 14.03s/it]


In [38]:
pmid_cuis_bench_cooc_df[['source','target']] = (
    pmid_cuis_bench_cooc_df['pair'].str.split(expand=True)
)

In [43]:
pmid_cuis_bench_cooc_df

,abstr_lvl_cooc,pair,date,from_ctd,from_gwas,from_drugcentral,from_mentha,from_rxnav,from_stringdb,from_kegg,from_disgennet,source,target
0,68,C0048581 C0134796,1975-11,False,False,False,False,False,False,True,False,C0048581,C0134796
0,68,C0017725 C0043047,1975-11,False,False,False,False,False,False,True,False,C0017725,C0043047
1,107,C0003818 C0302583,1975-10,False,False,False,False,False,False,True,False,C0003818,C0302583
1,107,C0009968 C0024706,1975-10,False,False,False,False,False,False,True,False,C0009968,C0024706
1,107,C0023175 C0043047,1975-10,False,False,False,False,False,False,True,False,C0023175,C0043047
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7202663,36475470,C0009241 C0338656,2022-12,False,False,False,False,False,False,False,True,C0009241,C0338656
7202663,36475470,C0038454 C0338656,2022-12,False,False,False,False,False,False,False,True,C0038454,C0338656
7202663,36475470,C0009241 C0948008,2022-12,False,False,False,False,False,False,False,True,C0009241,C0948008
7202663,36475470,C0038454 C0948008,2022-12,False,False,False,False,False,False,False,True,C0038454,C0948008


In [44]:
pmid_cuis_bench_cooc_df.to_pickle(
    '../benchmark_data/04_cross_ref_cui_pairs_pmid_ts/pmids_cooc_expd_df.pkl'
)

### Summarizing sources (for Table 1)

In [5]:
pmid_cuis_bench_cooc_df = pd.read_pickle(
    '../benchmark_data/04_cross_ref_cui_pairs_pmid_ts/pmids_cooc_expd_df.pkl'
)

In [13]:
source_cols_list = [c for c in pmid_cuis_bench_cooc_df.columns if 'from' in c]
source_cols_list

['from_ctd',
 'from_gwas',
 'from_drugcentral',
 'from_mentha',
 'from_rxnav',
 'from_stringdb',
 'from_kegg',
 'from_disgennet']

In [17]:
source_cols_names_dict = {
    'from_ctd': 'CTD',
    'from_gwas': 'GWAS',
    'from_drugcentral': 'DrugCentral',
    'from_mentha': 'Mentha',
    'from_rxnav': 'RxNav',
    'from_stringdb': 'STRING',
    'from_kegg': 'KEGG',
    'from_disgennet': 'DisGenNET'
}

In [28]:
pmid_cuis_bench_cooc_nopmid_df = (
    pmid_cuis_bench_cooc_df[['pair'] + source_cols_list]
        .drop_duplicates()
        .rename(columns=source_cols_names_dict)
)

In [32]:
db_to_n_nodes_dict = dict()

for db_source in tqdm(source_cols_names_dict.values()):
    db_pairs_list = pmid_cuis_bench_cooc_nopmid_df[
        pmid_cuis_bench_cooc_nopmid_df[db_source] == True
    ]['pair'].to_list()
    
    db_nodes_set = set()
    for pair in db_pairs_list:
        n1, n2 = pair.split(' ')
        db_nodes_set.add(n1)
        db_nodes_set.add(n2)
    db_to_n_nodes_dict[db_source] = len(db_nodes_set)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.90it/s]


In [33]:
db_to_n_nodes_dict

{'CTD': 37065,
 'GWAS': 8905,
 'DrugCentral': 19066,
 'Mentha': 10096,
 'RxNav': 2804,
 'STRING': 9118,
 'KEGG': 22306,
 'DisGenNET': 18923}

In [52]:
db_to_concept_types_dict = {
    'CTD': 'Genes, Diseases, Chemicals',
    'GWAS': 'Genes, Diseases',
    'DrugCentral': 'Genes, Diseases, Drugs',
    'Mentha': 'Proteins',
    'RxNav': 'Drugs',
    'STRING': 'Proteins',
    'KEGG': 'Genes, Diseases, Chemicals, Drugs',
    'DisGenNET': 'Genes, Diseases'
}

In [77]:
dbs_stat_counts_df = (
    pmid_cuis_bench_cooc_nopmid_df[source_cols_names_dict.values()]
        .sum(axis=0)
        .sort_values(ascending=False)
        .to_frame()
        .rename(columns={0:'Pairs $(m_i, m_j)$'})
)

In [78]:
dbs_stat_counts_df['Concepts $m_i$'] = (
    dbs_stat_counts_df
        .index
        .map(
            lambda x: db_to_n_nodes_dict.get(x)
        )
)

dbs_stat_counts_df['Concept Types'] = (
    dbs_stat_counts_df
        .index
        .map(
            lambda x: db_to_concept_types_dict.get(x)
        )
)

In [79]:
dbs_stat_counts_df.index.name = 'Database'

In [80]:
dbs_stat_counts_df

,"Pairs $(m_i, m_j)$",Concepts $m_i$,Concept Types
Database,,,
KEGG,730214,22306,"Genes, Diseases, Chemicals, Drugs"
CTD,459229,37065,"Genes, Diseases, Chemicals"
DisGenNET,274320,18923,"Genes, Diseases"
DrugCentral,230805,19066,"Genes, Diseases, Drugs"
RxNav,175186,2804,Drugs
STRING,63904,9118,Proteins
Mentha,43673,10096,Proteins
GWAS,30350,8905,"Genes, Diseases"


In [72]:
print(dbs_stat_counts_df.to_latex(escape=False))

\begin{tabular}{lrrl}
\toprule
{} &  Pairs $(m_i, m_j)$ &  Concepts $m_i$ &                      Concept Types \\
Database    &                     &                 &                                    \\
\midrule
KEGG        &              730214 &           22306 &  Genes, Diseases, Chemicals, Drugs \\
CTD         &              459229 &           37065 &         Genes, Diseases, Chemicals \\
DisGenNET   &              274320 &           18923 &                    Genes, Diseases \\
DrugCentral &              230805 &           19066 &             Genes, Diseases, Drugs \\
RxNav       &              175186 &            2804 &                              Drugs \\
STRING      &               63904 &            9118 &                           Proteins \\
Mentha      &               43673 &           10096 &                           Proteins \\
GWAS        &               30350 &            8905 &                    Genes, Diseases \\
\bottomrule
\end{tabular}



/tmp/ipykernel_41824/201573475.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dbs_stat_counts_df.to_latex(escape=False))
